In [ ]:
pip install transformers

     |████████████████████████████████| 2.5MB 4.3MB/s 
     |████████████████████████████████| 901kB 23.1MB/s 
     |████████████████████████████████| 3.3MB 36.5MB/s 


In [ ]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import pandas as pd
from nltk import word_tokenize
from tqdm import tqdm
tqdm.pandas()

from sklearn.utils import shuffle

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
annotations = pd.read_csv('/content/drive/MyDrive/BERT/combined_annotations.csv',sep=',')
#annotations = pd.read_csv('/content/drive/MyDrive/BERT/Finetuning BERT on EMSCAD/Conference/conference_data.csv',sep=',')
annotations = shuffle(annotations,random_state=123)
annotations.shape[0]

annotations = annotations.rename(columns={'recovered_gram':'candidate_skill'})
annotations.head()


,left_context,candidate_skill,right_context,label
12617,experience in case,management,and housing resources,0
17522,cope with fluctuation in workload effective,team working,skill to ensure constructive use,1
8132,coverage relative to,other public and,private school counterparts,0
15314,work in different way enable service user to l...,independent,lifestyle main duty and responsibility a proj...,0
2308,customer relations through proactive account,management,outstanding customer service and sales,0


In [ ]:
candidate_skill_split = annotations['candidate_skill'].str.split(' ',expand=True)
candidate_skill_split = candidate_skill_split.add_prefix('token_')
candidate_skill_split.head()

,token_0,token_1,token_2,token_3
12617,management,None,None,None
17522,team,working,None,None
8132,other,public,and,None
15314,independent,None,None,None
2308,management,None,None,None


In [ ]:
annotations = candidate_skill_split.join(annotations)
annotations.head()

,token_0,token_1,token_2,token_3,left_context,candidate_skill,right_context,label
12617,management,None,None,None,experience in case,management,and housing resources,0
17522,team,working,None,None,cope with fluctuation in workload effective,team working,skill to ensure constructive use,1
8132,other,public,and,None,coverage relative to,other public and,private school counterparts,0
15314,independent,None,None,None,work in different way enable service user to l...,independent,lifestyle main duty and responsibility a proj...,0
2308,management,None,None,None,customer relations through proactive account,management,outstanding customer service and sales,0


In [ ]:
annotations['token_0'] = annotations['token_0']+':'+annotations['label'].astype(str)
annotations['token_1'] = annotations['token_1']+':'+annotations['label'].astype(str)
annotations['token_2'] = annotations['token_2']+':'+annotations['label'].astype(str)
#annotations['token_3'] = annotations['token_3']+':'+annotations['label'].astype(str)
annotations.head()

,token_0,token_1,token_2,token_3,left_context,candidate_skill,right_context,label
12617,management:0,NaN,NaN,None,experience in case,management,and housing resources,0
17522,team:1,working:1,NaN,None,cope with fluctuation in workload effective,team working,skill to ensure constructive use,1
8132,other:0,public:0,and:0,None,coverage relative to,other public and,private school counterparts,0
15314,independent:0,NaN,NaN,None,work in different way enable service user to l...,independent,lifestyle main duty and responsibility a proj...,0
2308,management:0,NaN,NaN,None,customer relations through proactive account,management,outstanding customer service and sales,0


In [ ]:
annotations = annotations.fillna('')
#annotations['combined'] = annotations['left_context'] + ' ' + annotations['token_0'] + ' ' + annotations['token_1'] + ' '+ annotations['token_2'] + ' ' + annotations['token_3'] + ' ' + annotations['right_context']+'.'
annotations['combined'] = annotations['left_context'] + ' ' + annotations['token_0'] + ' ' + annotations['token_1'] + ' '+ annotations['token_2'] + ' ' + annotations['right_context']+'.'

annotations.head()

,token_0,token_1,token_2,token_3,left_context,candidate_skill,right_context,label,combined
12617,management:0,,,,experience in case,management,and housing resources,0,experience in case management:0 and housing ...
17522,team:1,working:1,,,cope with fluctuation in workload effective,team working,skill to ensure constructive use,1,cope with fluctuation in workload effective t...
8132,other:0,public:0,and:0,,coverage relative to,other public and,private school counterparts,0,coverage relative to other:0 public:0 and:0 pr...
15314,independent:0,,,,work in different way enable service user to l...,independent,lifestyle main duty and responsibility a proj...,0,work in different way enable service user to l...
2308,management:0,,,,customer relations through proactive account,management,outstanding customer service and sales,0,customer relations through proactive account m...


In [ ]:
#Breaking each sentence down into tokens for annotation

def tuple_transformer(dataset):
    token_df = pd.DataFrame()

    dataset= dataset

    for index in tqdm(range(0,len(dataset['combined']))):
        tokens = pd.DataFrame(word_tokenize(dataset['combined'].iloc[index]),columns=['tokens'])
        tokens['index'] = index

        token_df = pd.concat([token_df,tokens])

    annotationcolumn = token_df['tokens'].str.split(':',expand=True)

    token_df['annotation'] = annotationcolumn[1]
    token_df['tokens'] = annotationcolumn[0]

    token_df['annotation'] = token_df['annotation'].fillna('O')
    token_df.annotation[token_df.annotation=='0'] = 'O'
    token_df.annotation[token_df.annotation=='1'] = 'Soft_Skill'
    token_df.annotation[token_df.annotation=='2'] = 'Hard_Skill'

    token_df = token_df.rename(columns={'index':'sentence'})

    token_df['tuples'] = list(zip(token_df.tokens, token_df.annotation))

    tuple_list = []
    sublist=[]
    for row in tqdm(token_df['tuples']):
        sublist.append(row)
        if row[0]=='.':
            tuple_list.append(sublist)
            sublist=[]
            next

    return tuple_list

In [ ]:
#train_sents = tuple_transformer(train)
#test_sents = tuple_transformer(test)
nltk.download('punkt')
Complete_sents = tuple_transformer(annotations[['combined']])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


100%|██████████| 20674/20674 [01:22<00:00, 251.00it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 222304/222304 [00:00<00:00, 900538.69it/

In [ ]:
texts = []
tags = []

token_sublist = []
label_sublist = []

for sublist in tqdm(Complete_sents):
    for token,label in sublist:
        #print(token,label)
    
        token_sublist.append(token)
        label_sublist.append(label)
        if token == '.':            
            texts.append(token_sublist)
            tags.append(label_sublist)
            
            token_sublist = []
            label_sublist = []



100%|██████████| 20674/20674 [00:00<00:00, 190730.28it/s]


In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_tags, val_tags = train_test_split(texts, tags, test_size=.2)

In [ ]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
#from transformers import DistilBertTokenizerFast
#tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased',num_labels=3)
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

In [ ]:
import numpy as np

def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [ ]:
import tensorflow as tf

train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

In [ ]:
from transformers import TFBertForTokenClassification, BertForTokenClassification

model = TFBertForTokenClassification.from_pretrained('bert-base-uncased',num_labels=3)

All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss)
model.fit(train_dataset.shuffle(1000).batch(16), epochs=3, batch_size=16)

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

/usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:376: UserWarning: Using `-1` to mask the loss for the token is deprecated. Please use `-100` instead.
  return py_builtins.overload_of(f)(*args)


1034/1034 [==============================] - 194s 138ms/step - loss: 0.0864
Epoch 2/3
1034/1034 [==============================] - 145s 140ms/step - loss: 0.0474
Epoch 3/3
1034/1034 [==============================] - 145s 140ms/step - loss: 0.0329


In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.


In [ ]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.13.3).
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.


In [ ]:
!git config --global user.email "Ivowings"
!git config --global user.name "Ivo Wings"

In [ ]:
#!transformers-cli login

In [ ]:
token = 'uHuXixshJobFWRxPgsjJCLYbeYpTKCBWcLJnNlVUwpkeSRILcDlgiTbYrJhyfLDhFLHJzeBwBJNztbnPltCIzWIgLmQVujHdANVmIpiKLfZBvZUjKjCCuWXAkZwOgFgE'
#url = 'https://huggingface.co/Ivo/emscad-skill-extraction-conference-token-classification'
#short = 'Ivo/emscad-skill-extraction-conference-token-classification'

url = 'https://huggingface.co/Ivo/emscad-skill-extraction-token-classification'
short = 'Ivo/emscad-skill-extraction-token-classification'

model.save_pretrained('/content/drive/MyDrive/BERT/Finetuning BERT on EMSCAD/Finetuned model',
                       push_to_hub=True,
                       repo_url=url,
                       use_auth_token=token
                      )


In [ ]:
tokenizer.save_pretrained('/content/drive/MyDrive/BERT/Finetuning BERT on EMSCAD/Finetuned model',
                       push_to_hub=True,
                       repo_url=url,
                       use_auth_token=token
                      )

('/content/drive/MyDrive/BERT/Finetuning BERT on EMSCAD/Finetuned model/tokenizer_config.json',
 '/content/drive/MyDrive/BERT/Finetuning BERT on EMSCAD/Finetuned model/special_tokens_map.json',
 '/content/drive/MyDrive/BERT/Finetuning BERT on EMSCAD/Finetuned model/vocab.txt',
 '/content/drive/MyDrive/BERT/Finetuning BERT on EMSCAD/Finetuned model/added_tokens.json',
 '/content/drive/MyDrive/BERT/Finetuning BERT on EMSCAD/Finetuned model/tokenizer.json')

In [ ]:
from transformers import TFBertForTokenClassification, BertForTokenClassification
pt_model = BertForTokenClassification.from_pretrained(short, from_tf=True)
pt_model.save_pretrained('/content/drive/MyDrive/BERT/Finetuning BERT on EMSCAD/Fine tuned model pytoch',
                      push_to_hub=True,
                      repo_url=url,
                      use_auth_token=token
                      )

All TF 2.0 model weights were used when initializing BertForTokenClassification.

All the weights of BertForTokenClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForTokenClassification for predictions without further training.
